In [1]:
!pip install cflib

In [2]:
!pip install pynput

In [1]:
import tempfile
import os
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.positioning.motion_commander import MotionCommander
from pynq.overlays.base import BaseOverlay
from pynq.lib import Button
import logging
import time
import cv2
import numpy as np
from pynq.lib.video import VideoMode
from pynq.lib.video import PIXEL_RGB


# writable directory for the cache
cache_dir = tempfile.mkdtemp()
os.environ["CF2_CACHE_DIR"] = cache_dir

# Initialize the Crazyradio USB dongle
cflib.crtp.init_drivers()

# Create a Crazyflie object for communication
drone = Crazyflie()

# Connect to the Crazyflie drone
drone.open_link("radio://0/80/2M")

# Wait for the Crazyflie to be connected
while not drone.is_connected:
    print("Waiting for the Crazyflie to be connected")
    time.sleep(0.1)

if drone.is_connected:
    print("Crazyflie connected successfully!")
else:
    print("Failed to connect to the Crazyflie.")


Crazyflie connected successfully!


In [2]:
class ButtonDroneController:
    def __init__(self, cf, mc, buttons):
        self.cf = cf
        self.mc = mc
        self.buttons = buttons

        self.velocity = 0.5  # velocity value 
        self.fixed_height = 0.3  # height in meters
        self.taken_off = False
#Down
#Land
#Up
#Left 
#Right
        self.debounce_time = 10.0  # Debounce time in seconds
        self.last_button_press_time = [0] * len(buttons)  # Initialize last button press time for each button

    def control_drone(self, button):
        try:
            current_time = time.time()
            if current_time - self.last_button_press_time[button] < self.debounce_time:
                # Button press occurred within debounce time, ignore
                return

            self.last_button_press_time[button] = current_time
            
            if button == 0:  
                print('Landing...')
                button_press_time = time.time()  # Record button press time
                self.mc.land()
                self.taken_off = False  # Reset takeoff flag after landing
                command_execution_time = time.time()  # Record command execution time
                delay_time = command_execution_time - button_press_time
                print(f'Delay time: {delay_time:.2f} seconds')
            
            elif button == 1:  
                print('Up...')
                if not self.taken_off:
                    button_press_time = time.time()  
                    self.mc.take_off(self.fixed_height)
                    time.sleep(1.0)  #time.sleep(1.0)  # Add a delay after take-off for stabilization
                    self.taken_off = True
                    command_execution_time = time.time()  
                    delay_time = command_execution_time - button_press_time
                    print(f'Delay time: {delay_time:.2f} seconds')

                else:
                    self.mc.start_linear_motion(0, 0, 0, self.fixed_height)
                    button_press_time = time.time()  
                    self.mc.start_linear_motion(0, 0, self.velocity, self.fixed_height)
                    self.taken_off = True  # Reset takeoff flag after landing
                    command_execution_time = time.time()  
                    delay_time = command_execution_time - button_press_time
                    print(f'Delay time: {delay_time:.2f} seconds')

                    
            elif button == 2:  
                print('Down...')
                button_press_time = time.time()  
                self.mc.start_linear_motion(0, 0, -self.velocity, self.fixed_height)
                time.sleep(1.0)  
                self.taken_off = True 
                command_execution_time = time.time()  
                delay_time = command_execution_time - button_press_time
                print(f'Delay time: {delay_time:.2f} seconds')
                
            elif button == 3: 
                print('Left...')
                button_press_time = time.time()  
                self.mc.start_linear_motion(0, -self.velocity, 0, self.fixed_height)
                time.sleep(1.0)  
                self.mc.start_linear_motion(0, 0, 0, self.fixed_height)
                self.taken_off = True 
                command_execution_time = time.time()  
                delay_time = command_execution_time - button_press_time
                print(f'Delay time: {delay_time:.2f} seconds')
                
            elif button == 4: 
                print('Right...')
                button_press_time = time.time()  
                self.mc.start_linear_motion(0, self.velocity, 0, self.fixed_height)
                time.sleep(1.0)  
                self.mc.start_linear_motion(0, 0, 0, self.fixed_height)
                self.taken_off = True  
                command_execution_time = time.time()  
                delay_time = command_execution_time - button_press_time
                print(f'Delay time: {delay_time:.2f} seconds')

        
        except Exception as e:
            print(f"Exception in control_drone: {e}")



In [3]:

font = cv2.FONT_HERSHEY_PLAIN
frame_id = 1

#Initialize the BaseOverlay
base = BaseOverlay('base.bit')

buttons = [Button(base.buttons[i]) for i in range(1)]  # Assuming there are 4 buttons

# Initialize the MotionCommander with the Crazyflie object
mc = MotionCommander(drone)


# Gesture detection setup
net = cv2.dnn.readNet('yolov3_5_gestures.weights', 'yolov3_5_gestures.cfg')
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
    # Create a ButtonDroneController instance
controller = ButtonDroneController(drone, mc, classes)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode, PIXEL_RGB)
hdmi_out.start()

# Monitor (output) frame buffer size
frame_out_w = 640
frame_out_h = 480

# Camera (input) configuration
frame_in_w = 640
frame_in_h = 480

os.environ["OPENCV_LOG_LEVEL"]="SILENT"

# Initialize camera from HDMI input
videoIn = cv2.VideoCapture(0)  # Assuming HDMI input is connected to input port 0
#videoIn = cv2.cvtColor(cv2.COLOR_BGR2RGB)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);


print("Capture device is open: " + str(videoIn.isOpened()))

# Main loop for gesture detection and drone control
while True:
    starting_time = time.time()
    ret, frame = videoIn.read()
    if ret:
        height, width, channels = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (64, 64), (0, 0, 0), True, crop=False)

        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.80:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        label_detected = False
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                if label == "Up":
                    controller.control_drone(1)  # Trigger takeoff button
                elif label == "Land":
                    controller.control_drone(0)  # Trigger landing button
                elif label == "Down":
                    controller.control_drone(2)  # Trigger left button
                elif label == "Right":
                    controller.control_drone(4)  # Trigger Right button
                elif label == "Left":
                    controller.control_drone(3)  # Trigger Right button
                
                label_detected = True
                    
                confidence = confidences[i]
                color = colors[class_ids[i]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1, color,2)  
                elapsed_time = time.time() - starting_time
                fps = frame_id / elapsed_time
                cv2.putText(frame, "FPS: " + str(round(fps, 4)), (10, 50), font, 4, (0, 0, 0), 3)
                print(classes[class_ids[i]],fps)

        # Button-based control
        for i, button in enumerate(buttons):
            if button.read():  # Check if button is pressed
                controller.control_drone(i)

        
        if (ret):      
            outframe = hdmi_out.newframe()
            outframe[0:480,0:640,:] = frame[0:480,0:640,:]
            hdmi_out.writeframe(outframe)
    
        key = cv2.waitKey(1)
        if key == 27:
            break

# Release resources
videoIn.release()
cv2.destroyAllWindows()




Could not save cache, no writable directory
Could not save cache, no writable directory
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Capture device is open: True
Up...
Delay time: 3.61 seconds
Up 0.24080573707653283
Landing...
Delay time: 1.51 seconds
Up...
Delay time: 3.61 seconds
Up 0.2417639349168396
Up 2.1530726801861344
Landing...
Delay time: 1.51 seconds


KeyboardInterrupt: 